Let's query the CatSim database and pull down the stars in a given region and see what their spectra are.

Connect to fatboy with:

ssh -L 51433:fatboy.phys.washington.edu:1433 gateway.astro.washington.edu


If non-astro user, use simsuser@gateway.astro.washington.edu

Database schema at:  https://confluence.lsstcorp.org/display/SIM/Database+Schema

In [1]:
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.catUtils.baseCatalogModels import *
from lsst.sims.catUtils.exampleCatalogDefinitions import *
import numpy as np
from lsst.sims.utils import ObservationMetaData
import lsst.sims.photUtils.Sed as Sed
import lsst.sims.photUtils.Bandpass as Bandpass

/Users/yoachim/lsst/DarwinX86/anaconda/2.2.0/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/yoachim/lsst/DarwinX86/sims_catalogs_generation/sims_2.2.4/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/yoachim/lsst/DarwinX86/sims_catalogs_generation/sims_2.2.4/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [2]:
ra = 0.  # Degrees
dec = 0.  # Degrees
boundLength = 1. 
dbobj = CatalogDBObject.from_objid('allstars')
obs_metadata = ObservationMetaData(boundType='circle',pointingRA=ra,
                                   pointingDec=dec,boundLength=boundLength, mjd=5700)
t = dbobj.getCatalog('ref_catalog_star', obs_metadata=obs_metadata)

In [3]:
constraint = 'rmag < 18 and rmag > 15'
chunks = t.db_obj.query_columns(colnames=['magNorm', 'rmag', 'sedfilename', 'ebv', 'especid'], 
                                obs_metadata=obs_metadata,constraint=constraint, 
                                chunk_size=1000000)

/Users/yoachim/lsst/DarwinX86/sqlalchemy/2015_10.0-1-gaf5a1cb/lib/python/SQLAlchemy-1.0.8-py2.7-macosx-10.5-x86_64.egg/sqlalchemy/sql/elements.py:3779: SAWarning: Textual SQL expression 'rmag < 18 and rmag > 15' should be explicitly declared as text('rmag < 18 and rmag > 15') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [4]:
for chunk in chunks:
    catalog = chunk

In [5]:
print catalog

[ (426037734, 17.05723, ' lte035-5.0-1.0a+0.4.BT-Settl.spec.gz', 0.0323519, -1)
 (426038330, 16.99824, ' lte037-5.5-1.0a+0.4.BT-Settl.spec.gz', 0.0308979, -1)
 (426034893, 17.95176, ' lte034-4.5-1.0a+0.4.BT-Settl.spec.gz', 0.0388949, -1)
 ..., (818778168, 17.8869787, 'km10_4500.fits_g00_4700', 0.0298073, -1)
 (588510793, 15.8799033, 'km15_4250.fits_g00_4470', 0.0301708, -1)
 (818778743, 16.0920467, 'km10_5500.fits_g15_5720', 0.0301708, -1)]


In [17]:
# OK, now I'd like to load up the spectra of one of the stars and apply the extinction so it's the full SED as it would be observed in space
import os

# the SEDs are stored in sub-directories of the sims_sed_library home
# directory.  defaultSpecMap is a dict-like class that maps from the
# sed filenames stored in the CatSim database to the relevant
# sub-directory
from lsst.sims.utils import defaultSpecMap

# get the sims_sed_library home directory
from lsst.utils import getPackageDir
sed_dir = getPackageDir('sims_sed_library')

from lsst.sims.photUtils import Sed, Bandpass

In [18]:
imsimBand = Bandpass()
imsimBand.imsimBandpass()  # this is how we normalize SEDs

In [21]:
dex = 0 # index of the star whose spectrum we are generating

ss = Sed()
sed_name = os.path.join(sed_dir, defaultSpecMap[catalog['sedfilename'][dex]])
ss.readSED_flambda(sed_name)

# calculate the multiplicative normalization associated with 'magNorm'
# and multiply by that value.
# Note: if you wanted to normalize to a magnitude in a different band,
# you could do that by passing a different magnitude and bandpass
# into ss.calcFluxNorm()
fNorm = ss.calcFluxNorm(catalog['magNorm'][dex], imsimBand)
ss.multiplyFluxNorm(fNorm)

# set up some wavelength-dependent parameters for the dust model
# (note: even though the method is called CCMDust, this is actually
# applying the O'Donnell '94 dust model)
a_x, b_x = ss.setupCCMab()
ss.addCCMDust(a_x, b_x, ebv=catalog['ebv'][0])

# ss will have member variables wavelen (in nm) and flambda, which is
# ergs/s/cm^2/nm